In [7]:
%iam_role arn:aws:iam::116948010203:role/Poc-alertas-glue

%region us-east-1

%%tags
{
    "creado-por": "santiago.castro@externos-cl.cencosud.com",
    "apl": "apl1214",
    "unidad-negocio": "ccom",
    "bandera": "cencommerce",
    "plataforma": "eks",
    "version-so": "1.25",
    "Name": "CO-ALERT-POC",
    "ambiente": "staging",
    "cuenta": "116948010203",
    "pais": "co",
    "ceco": "CVO1007301",
    "Terraform": "no",
    "environment": "staging",
    "aplicacion": "co alert poc",
    "ApplicationName": "CO-ALERT-POC",
    "propietario": "humbertolares@cencosud.cl",
    "proyecto": "CENCO-PIM",
    "epm": "opex",
    "Owner": "Humberto Lares",
    "tribe": "Digital Retail Backbone"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.



Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 1.0.9 and you have 1.0.8 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Current iam_role is None
iam_role has been set to arn:aws:iam::116948010203:role/Poc-alertas-glue.
Previous region: us-east-1
Setting new region to: us-east-1
Region is set to: us-east-1
Tag {'creado-por': 'santiago.castro@externos-cl.cencosud.com', 'apl': 'apl1214', 'unidad-negocio': 'ccom', 'bandera': 'cencommerce', 'plataforma': 'eks', 'version-so': '1.25', 'Name': 'CO-ALERT-POC', 'ambiente': 'staging', 'cuenta': '116948010203', 'pais': 'co', 'ceco': 'CVO1007301', 'Terraform': 'no', 'environment': 'staging', 'aplicacion': 'co alert poc', 'ApplicationName': 'CO-ALERT-POC', 'propietario': 'humbertolares@cencos

In [ ]:
%idle_timeout 10
%glue_version 5.0
%worker_type G.1X
%number_of_workers 4

Current idle_timeout is None minutes.
idle_timeout has been set to 10 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 4


: 

: 

: 

In [ ]:
import sys
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.utils import getResolvedOptions

In [ ]:
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

In [ ]:
import os
import time
import logging
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from gcpspark import create_pyspark
from pyspark.sql.functions import col, current_date, date_sub

# Configurar logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Configurar logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Configuración de entorno
ENV = os.environ["ENVIRONMENT"]

properties = {
    "accesskey": os.getenv("colombia_sm_aws_access_key_id"),
    "secretkey": os.getenv("colombia_sm_aws_secret_access_key"),
    "sas_url": "",
    "sas": ""
}

In [ ]:
# Inicializar SparkSession con conexión a S3
spark_name = "data-extraction-br"
spark = create_pyspark(name=spark_name, connection="s3", properties=properties, verbose=True)

# --- Funciones de transformación ---

def transformation_stock(df):
    """Filtra stock del día anterior y unidades mayores a cero."""
    return (
        df.select("tienda_stock", "sku_stock", "ean_stock", "fecha_stock", "unidades_stock")
          .filter(col("fecha_stock") == date_sub(current_date(), 1))
          .filter(col("unidades_stock") > 0)
    )

def transformation_active(df):
    """Filtra productos activos ('estado_producto' == 'S')."""
    return df.filter(col("estado_producto") == "S").drop("estado_producto")

Maestra_Activa_Colombia* → BQ_CO_SM_ACTIVE_01

Maestra_Canal_Ventas_Colombia* → BQ_CO_SM_CHANNELS_01

Maestra_Promocion_Colombia* → BQ_CO_SM_PROMOTIONS_01

Maestra_Tienda_Colombia* → BQ_CO_SM_STORES_01

Productos_Arcus_Colombia* → BQ_CO_SM_PRODUCTS_01

Ventas_Arcus_Colombia* → BQ_CO_SM_SALES_01

Stock_Arcus_Colombia* → BQ_CO_SM_STOCK_01

Stock_Arcus_Colombia* → también se ejecuta semanalmente, además de diario

In [ ]:
# --- Configuración de fuentes ---

bucket_name = "cencosud.prod.sandbox.sm.col"

sources = [
    
    {
        "prefix": "ANALYTICS/ARCUS/Maestra_Activa_Colombia*",
        "bucket": f"{ENV}-bucket-dataproc-bigquery/dataproc/co_extraction",
        "table": "data_analytics_default.BQ_CO_SM_ACTIVE_01",
        "mode": "overwrite",
        "transformation": transformation_active
    },
    {
        "prefix": "ANALYTICS/ARCUS/Maestra_Canal_Ventas_Colombia*",
        "bucket": f"{ENV}-bucket-dataproc-bigquery/dataproc/co_extraction",
        "table": "data_analytics_default.BQ_CO_SM_CHANNELS_01",
        "mode": "overwrite"
    },
    {
        "prefix": "ANALYTICS/ARCUS/Maestra_Promocion_Colombia*",
        "bucket": f"{ENV}-bucket-dataproc-bigquery/dataproc/co_extraction",
        "table": "data_analytics_default.BQ_CO_SM_PROMOTIONS_01",
        "mode": "overwrite"
    },
    {
        "prefix": "ANALYTICS/ARCUS/Maestra_Tienda_Colombia*",
        "bucket": f"{ENV}-bucket-dataproc-bigquery/dataproc/co_extraction",
        "table": "data_analytics_default.BQ_CO_SM_STORES_01",
        "mode": "overwrite"
    },
    {
        "prefix": "ANALYTICS/ARCUS/Productos_Arcus_Colombia*",
        "bucket": f"{ENV}-bucket-dataproc-bigquery/dataproc/co_extraction",
        "table": "data_analytics_default.BQ_CO_SM_PRODUCTS_01",
        "mode": "append"
    },
    {
        "prefix": "ANALYTICS/ARCUS/Ventas_Arcus_Colombia*",
        "bucket": f"{ENV}-bucket-dataproc-bigquery/dataproc/co_extraction",
        "table": "data_analytics_default.BQ_CO_SM_SALES_01",
        "mode": "append"
    },
    {
        "prefix": "ANALYTICS/ARCUS/Stock_Arcus_Colombia*",
        "bucket": f"{ENV}-bucket-dataproc-bigquery/dataproc/co_extraction",
        "table": "data_analytics_default.BQ_CO_SM_STOCK_01",
        "mode": "overwrite",
        "transformation": transformation_stock
    }
]

In [ ]:
from datetime import datetime
today = datetime.today().weekday()  # 0 = lunes

# Tablas que solo se ejecutan los lunes
weekly_tables = {
    "data_analytics_default.BQ_CO_SM_ACTIVE_01",
    "data_analytics_default.BQ_CO_SM_CHANNELS_01",
    "data_analytics_default.BQ_CO_SM_PROMOTIONS_01",
    "data_analytics_default.BQ_CO_SM_STORES_01",
    "data_analytics_default.BQ_CO_SM_PRODUCTS_01"
}

In [ ]:
# --- Proceso por cada fuente ---

for source in sources:
    table_name = source["table"]
    
    if table_name in weekly_tables and today != 0:
        logging.info(f"Saltando tabla {table_name} (frecuencia semanal, hoy no es lunes).")
        continue

    prefix = source["prefix"]
    gcs_temp_bucket = source["bucket"]
    
    logging.info(f"Iniciando procesamiento para tabla: {table_name}")
    data_loaded = False
    retries = 17
    start_time = time.time()

    while not data_loaded and retries > 0:
        df = spark.read.parquet(f"s3a://{bucket_name}/{prefix}")

        # Aplicar transformación si existe
        if "transformation" in source:
            df = source["transformation"](df)

        # Validar si hay datos
        has_data = df.limit(1).count() > 0

        if has_data:
            data_loaded = True
            df = df.cache()  # cache si se va a usar varias veces
        else:
            logging.warning("No hay datos nuevos. Reintentando en 10 minutos...")
            retries -= 1
            time.sleep(600)

    if not data_loaded:
        logging.warning(f"No se encontró información para la tabla {table_name} tras múltiples intentos.")
        continue

    if source["mode"] == "overwrite":
        df.write.format("bigquery") \
            .option("temporaryGcsBucket", gcs_temp_bucket) \
            .option("table", table_name) \
            .mode("overwrite") \
            .save()
        logging.info(f"Datos sobreescritos en {table_name}")
    else:
        # Append condicional solo si hay datos nuevos únicos
        historical_df = spark.read.format("bigquery") \
            .option("table", table_name) \
            .load() \
            .drop("_PARTITIONTIME", "_PARTITIONDATE")

        new_unique_df = df.exceptAll(historical_df)
        new_count = new_unique_df.limit(1).count()

        if new_count == 0:
            logging.info(f"No se encontraron registros únicos nuevos para {table_name}.")
            continue

        new_unique_df.write.format("bigquery") \
            .option("temporaryGcsBucket", gcs_temp_bucket) \
            .option("table", table_name) \
            .mode("append") \
            .save()
        logging.info(f"{new_count} registros únicos agregados a {table_name}.")

    duration = round((time.time() - start_time)/60, 2)
    logging.info(f"Proceso completado para {table_name} en {duration} minutos.")

In [ ]:
# Cerrar sesión Spark
spark.stop()
logging.info("Proceso finalizado correctamente.")